Let's make this interface really simple for now and extensible for later. The main purpose of the interface is to make it easy to manually label our data. To this end, I will read in our JSON file continuously, record user information, and save it. There will be minimal error handling or edge case testing. We assume the user will select the proper results each time. User progress will not be saved. If overrides are needed, please override throught the interface code itself. This is be very jank, but should be fast, simple, and work for our very specific use case.

Make continuous annotation_ui file stream. We will assume that the user is keeping track of which annotations they've already made on files.

In [28]:
import gradio as gr
import re
import json
import pandas as pd

##########################################################################
### CONSTANTS -- QUESTIONS  
##########################################################################
q1_realistic = "Given the opening statement, is this question realistic for the justice to ask?"
q2_similarity = "Is this question similar in content to something the justice actually asked?"
q3_valence = "Is this question similar in valence/sentiment to the justice's actual questions?"
q4_helpfulness = "If you were preparing this opening statement, would you find this question helpful?"
q5_preference = "If you were preparing for oral arguments, would you prefer the model's question to the actual question?"

##########################################################################
### INPUT -- JSONs
##########################################################################
with open('datasets/2024_questions_for_eval.json') as f:
    all_data_to_annotate = json.load(f)

BRIEF_TRANSCRIPTS_DIR = "2023-2024_case_briefs/"
def get_brief_json(transcript_id):
    brief_name = transcript_id[:11] + ".json"
    with open(BRIEF_TRANSCRIPTS_DIR + brief_name) as json_file:
        brief_json = json.load(json_file)
    return brief_json

##########################################################################
### INPUT PARSING -- GENERATING MARKDOWN HELPERS
##########################################################################
def get_speaker(opening_statement):
    return re.findall(r"<speaker>(.*?)</speaker>", opening_statement)[0]

def get_opening_text(opening_statement):
    return re.findall(r"<text>(.*?)</text>", opening_statement)[0]

def get_text(case_metadata):
    return case_metadata.replace("<p>", "").replace("</p>", "").replace("\n", " ")

def format_opening_statement(opening_statement):
    return f"**{get_speaker(opening_statement)}** {get_opening_text(opening_statement)}"

def get_year(transcript_id):
    return transcript_id[:4]

def get_docket(transcript_id):
    return transcript_id[5:11]

##########################################################################
### INPUT DISPLAYING -- GENERATING MARKDOWN
##########################################################################
def generate_markdown(transcript_id, petitioner_or_respondent, justice_name):
    selected_data = [x for x in all_data_to_annotate if x["transcript_id"] == transcript_id][0]
    advocate_side = selected_data[petitioner_or_respondent.lower()]
    transcript_questions = advocate_side[("alito" if justice_name == "Samuel Alito" else "sotomayor")]["true_questions"]
    
    opening_statement = advocate_side["opening_statement"]
    coherent_questions = transcript_questions[0]["coherent"]
    brief_json = get_brief_json(transcript_id)

    return f"""
        # {brief_json["name"]} (Docket {get_docket(transcript_id)}, {get_year(transcript_id)})
        ### Facts of the Case
        {get_text(brief_json["facts_of_the_case"])}
        ### Legal Question
        {get_text(brief_json["question"])}
        ## {petitioner_or_respondent} {get_speaker(opening_statement)}'s Opening Statement
        {get_opening_text(opening_statement)}
        ## {justice_name}'s Questions (from the Oral Argument Transcripts)
        {"\\".join([f"* {get_text(x)}'" for x in coherent_questions])}
        ## LLM-as-{justice_name}'s Questions
        Each tab below contains a question generated by a LLM as {justice_name}. Please select the proper annotation
        for each survey question. Clicking "save" will add your annotation to our database and override any pre-existing 
        annotations.
        """

In [ ]:
##########################################################################
#### OUPUT LOADING -- PANDAS
##########################################################################
QID_DATA_DIR = "datasets/2024_questions/with_qid/"
FILE_TO_SAVE = "2024_llm_questions_Meta-Llama-3.1-70B-Instruct_qid_manual_labels.csv"
manual_annotations_df = pd.read_csv(QID_DATA_DIR + FILE_TO_SAVE)
manual_annotations_df.drop("Unnamed: 0", axis=1, inplace=True)

manual_annotation_types = ["is_realistic", "how_similar", "how_similar_sentiment", "how_helpful", "how_prefer_to_actual"]
for annotation_type in manual_annotation_types:
    if annotation_type not in manual_annotations_df:
        manual_annotations_df[annotation_type] = None
##########################################################################

In [62]:
def populate_llama_generated_questions(transcript_id, petitioner_or_respondent, justice_name):
    selected_transcript = [x for x in all_data_to_annotate if x["transcript_id"] == transcript_id][0]
    advocate_side = selected_transcript[petitioner_or_respondent.lower()]
    generated_questions = advocate_side[("alito" if justice_name == "Samuel Alito" else "sotomayor")]["predicted_questions"]
    llama_generated_questions = generated_questions[0]["model_questions"]
    return llama_generated_questions

def make_likert_scale(question, negation, property):
    label = f"{question} (-2 is \"Very {negation}\" and 2 is \"Very {property}\")"
    return gr.Radio(range(-2, 3, 1), label=label)

def save_annotations(predicted_question, is_realistic, how_similar, how_similar_sentiment, how_helpful, how_prefer):
    question_text = re.findall(r'"(.*?)"', predicted_question)[0]
    find_row_df_bool = manual_annotations_df['question_text'] == question_text
    if manual_annotations_df.loc[find_row_df_bool].empty:
        return "I'm sorry, but this question does not match our CSV. Please update the CSV and try again."
    # add in all given manual annotations
    manual_annotations_df.loc[find_row_df_bool, "is_realistic"] = is_realistic
    manual_annotations_df.loc[find_row_df_bool, "how_similar"] = how_similar
    manual_annotations_df.loc[find_row_df_bool, "how_similar_sentiment"] = how_similar_sentiment
    manual_annotations_df.loc[find_row_df_bool, "how_helpful"] = how_helpful
    manual_annotations_df.loc[find_row_df_bool, "how_prefer_to_actual"] = how_prefer
    return "Thank you for your annotation!"
    
with gr.Blocks(theme=gr.themes.Ocean()) as demo:
    generated_questions = gr.State([])
    with gr.Column() as config_setup_row:
        # initial config set-up
        transcript_select_dropdown = gr.Dropdown((x["transcript_id"] for x in all_data_to_annotate), 
                                                label="Select Transcript to Annotate", interactive=True)
        petitioner_or_respondent_select = gr.Radio(["Petitioner", "Respondent"], label="Select Side", interactive=True)
        justice_select = gr.Radio(["Sonia Sotomayor", "Samuel Alito"], label="Select Justice", interactive=True, visible=False)
        question_select_button = gr.Button(value="Annotate!")

    with gr.Column(visible=False) as case_info:
        transcript_info = gr.Markdown()
    
        @gr.render(inputs=generated_questions)
        def show_split(llm_questions):
            for i in range(len(llm_questions)):
                question_id = f"Question {i + 1}"
                with gr.Tab(question_id):
                    with gr.Column() as question_survey:
                        predicted_question = gr.Markdown(f"**Predicted Question**: \"{llm_questions[i]}\"")
                        is_realistic = gr.Radio(["Yes", "No"], label=q1_realistic)
                        how_similar = make_likert_scale(q2_similarity, "Similar", "Dissimilar")
                        how_similar_sentiment = make_likert_scale(q3_valence, "Dissimilar in Sentiment", "Similar in Sentiment")
                        how_helpful = make_likert_scale(q4_helpfulness, "Unhelpful", "Helpful")
                        how_prefer = make_likert_scale(q5_preference, "Prefer the Actual Question", "Prefer the Model Question")
                    
                    on_submission_text = gr.Markdown(visible=False)
                    submit_button = gr.Button(value=f"Submit {question_id}")
                    submit_button.click(save_annotations, [predicted_question, is_realistic, how_similar, how_similar_sentiment, how_helpful, how_prefer], on_submission_text).then(
                        fn=lambda: [gr.Markdown(visible=True), gr.Column(visible=False), gr.Button(visible=False)], inputs=None, outputs=[on_submission_text, question_survey, submit_button]
                    )

    # config set-up interactive logic
    petitioner_or_respondent_select.select(fn=lambda: gr.Radio(visible=True), inputs=None, outputs=justice_select)
    justice_select.select(fn=lambda: gr.Button(visible=True), inputs=None, outputs=question_select_button)
    question_select_button.click(fn=lambda: [gr.Row(visible=False), gr.Column(visible=True)], inputs=None, outputs=[config_setup_row, case_info]).then(
        generate_markdown, [transcript_select_dropdown, petitioner_or_respondent_select, justice_select], transcript_info
    ).then(
        populate_llama_generated_questions, [transcript_select_dropdown, petitioner_or_respondent_select, justice_select], generated_questions
    )
demo.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [63]:
demo.close()

Closing server running on port: 7866


In [64]:
manual_annotations_df

,Unnamed: 0,question_id,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,is_realistic,how_similar,how_similar_sentiment,how_helpful,how_prefer_to_actual
0,0,q_577c46f5,2024.23-621-t01,petitioner,sotomayor,Can you explain why you believe the Court shou...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",Yes,0,-1,1,-1
1,1,q_4eb90a53,2024.23-621-t01,petitioner,sotomayor,How do you respond to the concern that your pr...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",No,-1,1,2,-1
2,2,q_ef2ef104,2024.23-621-t01,petitioner,sotomayor,You rely on legal dictionaries from the time o...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",None,None,None,None,None
3,3,q_ff25bfb9,2024.23-621-t01,petitioner,sotomayor,Don't some of this Court's precedents suggest ...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",Yes,0,-1,2,1
4,4,q_3be72c20,2024.23-621-t01,petitioner,sotomayor,Can you think of any situations in which a pre...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",Yes,0,-1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,306,q_ef8e287d,2024.23-713-t01,respondent,alito,Isn't it true that your proposed bright-line r...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,None,None,None,None,None
307,307,q_cc54a35b,2024.23-713-t01,respondent,alito,How do you respond to the argument that the te...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,None,None,None,None,None
308,308,q_a701dfc3,2024.23-713-t01,respondent,alito,Can you reconcile your reliance on historical ...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,None,None,None,None,None
309,309,q_c63cea72,2024.23-713-t01,respondent,alito,Would your interpretation not create perverse ...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,<speaker>Sopan Joshi</speaker><text> Mr. Chief...,None,None,None,None,None
